In [1]:
import pandas as pd

df = pd.read_csv("../data/raw/Goa real estate.csv")

df.head()


,Location,BHK,price,sq_feet,latitude,longitude
0,Mapusa,Office Space,100000,77.0,15.590853,73.810215
1,Taleigao,2 BHK,10000000,11905.0,15.470266,73.822567
2,Calangute,2 BHK,10000000,12500.0,15.545594,73.764618
3,Tuem,Land,10000000,2714.0,15.666568,73.795665
4,Bainguinim,2 BHK,10000000,7492.0,15.500349,73.902288


In [2]:
# Clean column names safely
df.columns = (
    df.columns
    .astype(str)
    .str.strip()
    .str.lower()
    .str.replace(" ", "_")
    .str.replace("/", "_")
    .str.replace(".", "")
)

df.head()


,location,bhk,price,sq_feet,latitude,longitude
0,Mapusa,Office Space,100000,77.0,15.590853,73.810215
1,Taleigao,2 BHK,10000000,11905.0,15.470266,73.822567
2,Calangute,2 BHK,10000000,12500.0,15.545594,73.764618
3,Tuem,Land,10000000,2714.0,15.666568,73.795665
4,Bainguinim,2 BHK,10000000,7492.0,15.500349,73.902288


In [3]:
# Convert price and sq_feet to numeric
df['price'] = pd.to_numeric(df['price'], errors='coerce')
df['sq_feet'] = pd.to_numeric(df['sq_feet'], errors='coerce')

# Remove rows with missing values
df = df.dropna(subset=['price', 'sq_feet'])

df.head()


,location,bhk,price,sq_feet,latitude,longitude
0,Mapusa,Office Space,100000,77.0,15.590853,73.810215
1,Taleigao,2 BHK,10000000,11905.0,15.470266,73.822567
2,Calangute,2 BHK,10000000,12500.0,15.545594,73.764618
3,Tuem,Land,10000000,2714.0,15.666568,73.795665
4,Bainguinim,2 BHK,10000000,7492.0,15.500349,73.902288


In [4]:
df['bhk_num'] = df['bhk'].astype(str).str.extract(r'(\d+)').astype(float)
df['bhk_num'] = df['bhk_num'].fillna(0)

df[['bhk', 'bhk_num']].head()


,bhk,bhk_num
0,Office Space,0.0
1,2 BHK,2.0
2,2 BHK,2.0
3,Land,0.0
4,2 BHK,2.0


In [5]:
# Keep only reasonable price & area ranges
df = df[df['price'] < 5e8]        # below 50 crore
df = df[df['sq_feet'] < 50000]    # below 50,000 sqft

df.shape


(2428, 7)

In [6]:
sample = df.sample(40, random_state=42)
sample.to_csv("../data/sample/goa_sample.csv", index=False)

print("Sample saved to data/sample/goa_sample.csv")


Sample saved to data/sample/goa_sample.csv


In [7]:
import os

# create processed folder if not exists
os.makedirs("../data/processed", exist_ok=True)

df.to_csv("../data/processed/goa_cleaned.csv", index=False)
print("Cleaned dataset saved.")


Cleaned dataset saved.


In [8]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Load cleaned data
df = pd.read_csv("../data/processed/goa_cleaned.csv")
print("shape:", df.shape)
df.head()


shape: (2428, 7)


,location,bhk,price,sq_feet,latitude,longitude,bhk_num
0,Mapusa,Office Space,100000,77.0,15.590853,73.810215,0.0
1,Taleigao,2 BHK,10000000,11905.0,15.470266,73.822567,2.0
2,Calangute,2 BHK,10000000,12500.0,15.545594,73.764618,2.0
3,Tuem,Land,10000000,2714.0,15.666568,73.795665,0.0
4,Bainguinim,2 BHK,10000000,7492.0,15.500349,73.902288,2.0


In [9]:
# Basic stats
display(df[['price','sq_feet','bhk_num']].describe().T)

# Price per sqft
df['price_per_sqft'] = df['price'] / df['sq_feet']
display(df['price_per_sqft'].describe().T)


,count,mean,std,min,25%,50%,75%,max
price,2428.0,3.392160e+07,4.492240e+07,100000.0,7800000.00,15500000.0,47500000.00,450000000.0
sq_feet,2428.0,1.129141e+04,7.318026e+03,0.0,5962.75,9734.0,15183.75,49691.0
bhk_num,2428.0,2.082784e+00,1.545673e+00,0.0,1.00,2.0,3.00,10.0


c:\Users\Aditya\AppData\Local\Programs\Python\Python312\Lib\site-packages\pandas\core\nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)


count    2428.000000
mean             inf
std              NaN
min        48.981944
25%      1032.444139
50%      1694.722804
75%      3767.543163
max              inf
Name: price_per_sqft, dtype: float64

In [10]:
# Remove rows with sq_feet = 0 or missing (these break price_per_sqft)
before = df.shape[0]
df = df[df['sq_feet'].notna() & (df['sq_feet'] > 0)]
after = df.shape[0]

print(f"Removed {before - after} bad rows (sq_feet <= 0). Remaining rows: {after}")


Removed 1 bad rows (sq_feet <= 0). Remaining rows: 2427


In [11]:
import numpy as np

df['price_per_sqft'] = df['price'] / df['sq_feet']

# Replace infinities (if any)
df['price_per_sqft'].replace([np.inf, -np.inf], np.nan, inplace=True)

# Drop rows where price_per_sqft is NaN
before2 = df.shape[0]
df = df.dropna(subset=['price_per_sqft'])
after2 = df.shape[0]

print(f"Dropped {before2 - after2} rows with invalid price_per_sqft")


Dropped 0 rows with invalid price_per_sqft


C:\Users\Aditya\AppData\Local\Temp\ipykernel_23180\756004707.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['price_per_sqft'].replace([np.inf, -np.inf], np.nan, inplace=True)


In [12]:
df['price_per_sqft'].describe()


count    2.427000e+03
mean     1.041577e+04
std      7.755150e+04
min      4.898194e+01
25%      1.031617e+03
50%      1.694448e+03
75%      3.767519e+03
max      1.963190e+06
Name: price_per_sqft, dtype: float64

In [13]:
# Cap extreme price_per_sqft values at 99.5 percentile
p995 = df['price_per_sqft'].quantile(0.995)
df = df[df['price_per_sqft'] <= p995]

print("New shape after capping:", df.shape)
df['price_per_sqft'].describe()


New shape after capping: (2414, 8)


count      2414.000000
mean       5648.333183
std       22848.816288
min          48.981944
25%        1023.248495
50%        1688.499181
75%        3709.470577
max      398406.374502
Name: price_per_sqft, dtype: float64

In [14]:
df.to_csv("../data/processed/goa_cleaned.csv", index=False)
print("Saved cleaned dataset.")


Saved cleaned dataset.
